# Notebook that does the processing of foraging session trajectories
'''Documentation about the task here'''

### Imports  
All functions are in behavior_analysis_functions.py

In [1]:
import os
import gc
import glob
import time
from processing_TowerCoordinates import *
from processing_session_trajectory import *
# from functions_for_PDF import *


### Get mice data
Define the folder where your MOUXXX folder are, generate a list of MOUXXX folders and for each mice a list of sessions

In [2]:
# indicate where the data are
# path_to_data_folder is the path of the folder where you store your different mice.

path_to_data_folder = '/media/david/Maud_T5/Thesis/HarddriveDatas/Maud_Gq_BL'

# If you want to process all the mice in your data folder
pattern_of_MOU_Folders = os.path.join(path_to_data_folder, "MOU*")
mice_to_process = [os.path.basename(path) for path in glob.glob(pattern_of_MOU_Folders)]
mice_to_process=sorted(mice_to_process)

# Print the number and the list of mice
print(f'Found {len(mice_to_process)} {"mice" if len(mice_to_process) > 1 else "mouse"} in the data folder: {", ".join(mice_to_process)}\n')


#If you want to process a subset of mice uncomment the line below and write your mice names
# mice_to_process=["MOU3867","MOU4551","MOU4623","MOU4562","MOU4560","MOU4623","MOU4624","MOU4552","MOU4563","MOU4561","MOU4692","MOU4694","MOU4695","MOU4696","MOU4698","MOU4703","MOU4704","MOU4175","MOU4185"]
# mice_to_process = ["MOU21_1","MOU21_2","MOU21_3","MOU21_4","MOU21_5","MOU21_6"]
mice_to_process = ["MOU5395","MOU5396", "MOU5372", "MOU5373", "MOU5385", "MOU5343", "MOU5344", "MOU5387", "MOU5388"]
# mice_to_process = ["MOU5025","MOU5027","MOU5017","MOU5018","MOU5019","MOU5037","MOU5038","MOU5039"]
sessions_list = {}

for mouse in mice_to_process:

    mouse_folder = os.path.join(path_to_data_folder,mouse)
    sessions_list[mouse] = sorted([name for name in os.listdir(mouse_folder)
                           if os.path.isdir(os.path.join(mouse_folder, name))
                           and name.startswith('MOU')])
    nb_sessions = len(sessions_list[mouse])
    print(f'Hello, I\'m {mouse}! I have foraged for {nb_sessions} sessions:')
    print(sessions_list[mouse], '\n')


Found 9 mice in the data folder: MOU5343, MOU5344, MOU5372, MOU5373, MOU5385, MOU5387, MOU5388, MOU5395, MOU5396

Hello, I'm MOU5395! I have foraged for 30 sessions:
['MOU5395_20251003-1014', 'MOU5395_20251003-1512', 'MOU5395_20251006-1012', 'MOU5395_20251006-1546', 'MOU5395_20251007-1029', 'MOU5395_20251007-1553', 'MOU5395_20251008-1003', 'MOU5395_20251008-1509', 'MOU5395_20251009-0902', 'MOU5395_20251009-1555', 'MOU5395_20251010-0903', 'MOU5395_20251010-1410', 'MOU5395_20251011-0908', 'MOU5395_20251011-1441', 'MOU5395_20251012-0900', 'MOU5395_20251012-1400', 'MOU5395_20251013-0854', 'MOU5395_20251013-1359', 'MOU5395_20251014-0923', 'MOU5395_20251014-1429', 'MOU5395_20251015-0852', 'MOU5395_20251015-1403', 'MOU5395_20251016-0945', 'MOU5395_20251016-1448', 'MOU5395_20251017-1021', 'MOU5395_20251017-1521', 'MOU5395_20251018-0851', 'MOU5395_20251018-1349', 'MOU5395_20251019-0914', 'MOU5395_20251019-1414'] 

Hello, I'm MOU5396! I have foraged for 30 sessions:
['MOU5396_20251003-1034', 'MO

## Process all the sessions of the mice in the mice_to_process list

The 'overwrite' option can force processing. Set it to True if you want to process all sessions and False in case the data have already been processed and you don't want it to force it. 


In [3]:
overwrite = False  # Set to True if you want to overwrite existing pickle files
bad_sessions = []

for mouse in mice_to_process:

    mouse_folder = os.path.join(path_to_data_folder, mouse)

    # Get the list of sessions
    sessions_to_process = sorted([name for name in os.listdir(mouse_folder)
                                  if os.path.isdir(os.path.join(mouse_folder, name))
                                  and name.startswith('MOU')])

    nb_sessions = len(sessions_to_process)
    print(f'Processing mouse {mouse}. There is/are {nb_sessions} sessions to process:')
    print(sessions_to_process, '\n')

    # Process each session
    for sessionindex, session_to_process in enumerate(sessions_to_process):
        print(f'Processing the trajectory of session {session_to_process}')

        if sessionindex==0: # We get the trapeze coordinates from the 1st session of 1 animal, and we will resuse them for the remaining sessions
            trapeze_width, towers_coordinates = get_trapeze_and_tower_data(mouse_folder, session_to_process)
            all_trapezes_coordinates_cm,towers_coordinates_cm= generate_trapeze_and_tower_coordinates(towers_coordinates, trapeze_width)

        # Define the pickle file path
        output_pickle_filename = f"{session_to_process}_basic_processing_output.pickle"
        output_pickle_filepath = os.path.join(mouse_folder, session_to_process, output_pickle_filename)

        # Check if the pickle file already exists
        if not overwrite and os.path.exists(output_pickle_filepath):
            # print(f'Pickle file already exists for session {session_to_process}, skipping processing.')
            continue  # Skip processing if the file exists and overwrite is False

        # Run the processing if file doesn't exist or overwrite is True
        output=process_trajectory(mouse_folder, session_to_process,all_trapezes_coordinates_cm,towers_coordinates_cm)
        if output is not None:
            bad_sessions.append(output)
        print('#########################\n')


Processing mouse MOU5395. There is/are 30 sessions to process:
['MOU5395_20251003-1014', 'MOU5395_20251003-1512', 'MOU5395_20251006-1012', 'MOU5395_20251006-1546', 'MOU5395_20251007-1029', 'MOU5395_20251007-1553', 'MOU5395_20251008-1003', 'MOU5395_20251008-1509', 'MOU5395_20251009-0902', 'MOU5395_20251009-1555', 'MOU5395_20251010-0903', 'MOU5395_20251010-1410', 'MOU5395_20251011-0908', 'MOU5395_20251011-1441', 'MOU5395_20251012-0900', 'MOU5395_20251012-1400', 'MOU5395_20251013-0854', 'MOU5395_20251013-1359', 'MOU5395_20251014-0923', 'MOU5395_20251014-1429', 'MOU5395_20251015-0852', 'MOU5395_20251015-1403', 'MOU5395_20251016-0945', 'MOU5395_20251016-1448', 'MOU5395_20251017-1021', 'MOU5395_20251017-1521', 'MOU5395_20251018-0851', 'MOU5395_20251018-1349', 'MOU5395_20251019-0914', 'MOU5395_20251019-1414'] 

Processing the trajectory of session MOU5395_20251003-1014
Processing the trajectory of session MOU5395_20251003-1512
Processing the trajectory of session MOU5395_20251006-1012
Process

## Create PDFs of all sessions

Here you can choose to create a PDF (or not) in order to visualize the datas of all sessions. 

You can create 2 independant PDF : one containing all main metrics of different types of runs ; one containing the raster plot of visits and 2 distribution charts (rewarded visits duration, and number of runs in rewarded visits).

You can set the variables containing 'overwrite' to True if you want to recreate the PDF from the beginning (and therefore processing all sessions) or to add only new sessions that have not been plotted yet. You also have the possibility to generate only one PDF out of the two possible.

In [4]:
--------------------------------------

# Choose if you want to generate and/or overwrite the metrics PDF
generate_pdf_metrics = True
overwrite_pdf_metrics = False
output_path_metrics = f"{mouse_folder}/{mouse}_metrics_figures.pdf"

# Choose if you want to generate and/or overwrite the visits PDF
generate_pdf_visits = True
overwrite_pdf_visits = False 
output_path_visits = f"{mouse_folder}/{mouse}_visits_figures.pdf"

# Processing of sessions to create PDFs

for mouse in mice_to_process:

    print(f"Creating {mouse} PDFs\n")    
    mouse_folder = os.path.join(path_to_data_folder, mouse) 
    sessions_list = sorted([name for name in os.listdir(mouse_folder)
                if os.path.isdir(os.path.join(mouse_folder, name))
                and name.startswith('MOU')])
    
    if generate_pdf_metrics:
        print("Metrics PDF generation ongoing...\n")
        make_pdf_metrics(mouse_folder, sessions_list, mouse, output_path_metrics, overwrite=overwrite_pdf_metrics)

    if generate_pdf_visits: 
        print("\nVisits PDF generation ongoing...")
        make_pdf_visits(mouse_folder, sessions_list, mouse, output_path_visits, overwrite=overwrite_pdf_visits)

SyntaxError: invalid syntax (4220317535.py, line 1)